In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model

import scipy
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from sklearn import metrics
from collections import OrderedDict

In [76]:
data = torch.load('../savedir/CIFAR_ddb_fas_cummul_highfreq_remove.pth', map_location='cpu')
# no_img_df = {'ddbs': data['ddbs'], 'FAS_score': data['FAS_score'], 'steps_to_atk': data['steps_to_atk'], 'predictions': data['predictions'] }
df = pd.DataFrame(data)
df['ddbs'] = list(map(lambda x:float(x), df['ddbs']))
df.head()

,ddbs,gt,steps_to_atk,predictions,Flipping_Freq,Model_preds,lip
0,1.498108,1,2,9,18,1,13.673544
1,1.523164,2,1,3,17,2,10.107368
2,1.470038,0,1,2,5,0,12.825120
3,1.616426,9,1,1,18,9,9.064425
4,1.525899,0,1,2,17,0,8.146470


### Regression Analysis 1:
- Input: ddbs
- Output: lip

In [77]:

store_results = OrderedDict()
met = [metrics.explained_variance_score, metrics.max_error, metrics.mean_absolute_error, metrics.mean_squared_error, metrics.mean_squared_log_error, metrics.median_absolute_error, metrics.r2_score, metrics.mean_poisson_deviance, metrics.mean_gamma_deviance, metrics.mean_absolute_percentage_error]
for m in met:
    store_results[m.__name__] = []

for loss in met:
    try:

        ## Input: steps_to_atk
        ## Output: lip

        X = df['steps_to_atk'].values.reshape(-1,1)
        y = df['lip']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: ddbs
        ## Output: Lip

        X = df['ddbs'].values.reshape(-1,1)
        y = df['lip']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: Flipping_Freq
        ## Output: lip

        X = df['Flipping_Freq'].values.reshape(-1,1)
        y = df['lip']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)

        store_results[loss.__name__].append(loss(y_test,y_pred))


        ## Input: ddbs, Flipping_Freq
        ## Output: lip

        X = df[['ddbs','Flipping_Freq']]
        y = df['lip']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: steps_to_atk, Flipping_Freq
        ## Output: lip

        X = df[['steps_to_atk','Flipping_Freq']]
        y = df['lip']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

    except Exception as e:
        del store_results[loss.__name__]
        # print(f'skipped loss: {loss.__name__}')
        
store_results = pd.DataFrame(store_results)
store_results.index = ['steps_to_atk vs LIP', 'ddbs vs LIP', 'Flipping_Freq vs LIP','ddbs + Flipping_Freq vs LIP', 'steps_to_atk + Flipping_Freq vs LIP']
store_results

,explained_variance_score,max_error,mean_absolute_error,mean_squared_error,median_absolute_error,r2_score,mean_absolute_percentage_error
steps_to_atk vs LIP,0.055901,42.477095,4.032898,27.401150,3.329734,0.055885,0.340581
ddbs vs LIP,0.209930,36.803056,3.686062,22.930930,3.009845,0.209908,0.310265
Flipping_Freq vs LIP,-0.000061,42.528506,4.174653,29.025162,3.480962,-0.000071,0.355365
ddbs + Flipping_Freq vs LIP,0.211904,36.896890,3.679355,22.873672,2.996839,0.211881,0.309559
steps_to_atk + Flipping_Freq vs LIP,0.059838,42.210788,4.028327,27.286858,3.331420,0.059823,0.340227


In [91]:
data = torch.load('../savedir/CIFAR_ddb_fas_cummul_highfreq_remove.pth', map_location='cpu')
# no_img_df = {'ddbs': data['ddbs'], 'FAS_score': data['FAS_score'], 'steps_to_atk': data['steps_to_atk'], 'predictions': data['predictions'] }
df = pd.DataFrame(data)
df['ddbs'] = list(map(lambda x:float(x), df['ddbs']))

store_results = OrderedDict()
met = [metrics.explained_variance_score, metrics.max_error, metrics.mean_absolute_error, metrics.mean_squared_error, metrics.mean_squared_log_error, metrics.median_absolute_error, metrics.r2_score, metrics.mean_poisson_deviance, metrics.mean_gamma_deviance, metrics.mean_absolute_percentage_error]
for m in met:
    store_results[m.__name__] = []

for loss in met:
    try:

        ## Input: steps_to_atk
        ## Output: lip

        # X = df['steps_to_atk'].values.reshape(-1,1)
        # y = df['steps_to_atk']

        # X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        # reg = linear_model.LinearRegression()
        # reg.fit(X_train, y_train)
        # y_pred = reg.predict(X_test)
        # store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: ddbs
        ## Output: Lip

        X = df['ddbs'].values.reshape(-1,1)
        y = df['steps_to_atk']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: Flipping_Freq
        ## Output: lip

        X = df['Flipping_Freq'].values.reshape(-1,1)
        y = df['steps_to_atk']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)

        store_results[loss.__name__].append(loss(y_test,y_pred))


        ## Input: ddbs, Flipping_Freq
        ## Output: lip

        X = df[['ddbs','Flipping_Freq']]
        y = df['steps_to_atk']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: steps_to_atk, Flipping_Freq
        ## Output: lip

        X = df[['steps_to_atk','Flipping_Freq']]
        y = df['steps_to_atk']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

    except Exception as e:
        del store_results[loss.__name__]
        # print(f'skipped loss: {loss.__name__}')
        
store_results = pd.DataFrame(store_results)
store_results.index = ['ddbs vs steps_to_atk', 'Flipping_Freq vs steps_to_atk','ddbs + Flipping_Freq vs steps_to_atk', 'steps_to_atk + Flipping_Freq vs steps_to_atk']
store_results

,explained_variance_score,max_error,mean_absolute_error,mean_squared_error,median_absolute_error,r2_score,mean_absolute_percentage_error
ddbs vs steps_to_atk,0.067133,1.072250e+01,4.183962e-01,4.177997e-01,1.903613e-01,0.067124,6.903133e+14
Flipping_Freq vs steps_to_atk,0.074843,1.078486e+01,4.189193e-01,4.143471e-01,2.151408e-01,0.074833,6.409547e+14
ddbs + Flipping_Freq vs steps_to_atk,0.155778,1.050186e+01,4.233974e-01,3.780967e-01,2.690377e-01,0.155774,5.941143e+14
steps_to_atk + Flipping_Freq vs steps_to_atk,1.000000,2.309264e-13,8.014608e-15,1.991729e-28,8.881784e-16,1.000000,1.530899e+01


# Tiny ImageNet

In [92]:
data = torch.load('../savedir/TIMGNET_ddb_fas_cummul_highfreq_remove.pth', map_location='cpu')
# no_img_df = {'ddbs': data['ddbs'], 'FAS_score': data['FAS_score'], 'steps_to_atk': data['steps_to_atk'], 'predictions': data['predictions'] }
df = pd.DataFrame(data)
df['ddbs'] = list(map(lambda x:float(x), df['ddbs']))
df

,ddbs,gt,steps_to_atk,predictions,x,Flipping_Freq,Model_preds,lip
0,2.980269,65,1,5,0,47,65,84.094421
1,3.016810,166,1,133,1,51,166,34.080173
2,3.016907,4,1,134,2,47,4,56.624294
3,2.917111,196,1,17,3,37,196,83.534866
4,2.847197,75,1,156,4,33,75,68.558495
...,...,...,...,...,...,...,...,...
99995,2.921618,107,1,94,99995,34,107,50.238590
99996,2.609845,45,1,42,99996,50,45,66.069839
99997,3.066802,93,2,114,99997,43,93,69.157906
99998,2.813104,5,1,75,99998,46,5,66.551796


In [74]:
store_results = OrderedDict()
met = [metrics.explained_variance_score, metrics.max_error, metrics.mean_absolute_error, metrics.mean_squared_error, metrics.mean_squared_log_error, metrics.median_absolute_error, metrics.r2_score, metrics.mean_poisson_deviance, metrics.mean_gamma_deviance, metrics.mean_absolute_percentage_error]
for m in met:
    store_results[m.__name__] = []

for loss in met:
    try:

        ## Input: steps_to_atk
        ## Output: lip

        X = df['steps_to_atk'].values.reshape(-1,1)
        y = df['lip']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: ddbs
        ## Output: Lip

        X = df['ddbs'].values.reshape(-1,1)
        y = df['lip']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: Flipping_Freq
        ## Output: lip

        X = df['Flipping_Freq'].values.reshape(-1,1)
        y = df['lip']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)

        store_results[loss.__name__].append(loss(y_test,y_pred))


        ## Input: ddbs, Flipping_Freq
        ## Output: lip

        X = df[['ddbs','Flipping_Freq']]
        y = df['lip']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: steps_to_atk, Flipping_Freq
        ## Output: lip

        X = df[['steps_to_atk','Flipping_Freq']]
        y = df['lip']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

    except Exception as e:
        del store_results[loss.__name__]
        # print(f'skipped loss: {loss.__name__}')
        
store_results = pd.DataFrame(store_results)
store_results.index = ['steps_to_atk vs LIP', 'ddbs vs LIP', 'Flipping_Freq vs LIP','ddbs + Flipping_Freq vs LIP', 'steps_to_atk + Flipping_Freq vs LIP']
store_results

,explained_variance_score,max_error,mean_absolute_error,mean_squared_error,median_absolute_error,r2_score,mean_absolute_percentage_error
steps_to_atk vs LIP,0.071513,116.493527,11.140338,201.810782,9.333000,0.071499,0.211597
ddbs vs LIP,0.043547,103.331610,11.345827,207.888601,9.474847,0.043536,0.218574
Flipping_Freq vs LIP,0.003685,106.979778,11.607513,216.551552,9.743515,0.003679,0.222992
ddbs + Flipping_Freq vs LIP,0.053460,103.387959,11.272679,205.733363,9.400777,0.053452,0.216978
steps_to_atk + Flipping_Freq vs LIP,0.071761,115.230914,11.138369,201.756760,9.317603,0.071748,0.211555


In [93]:
data = torch.load('../savedir/TIMGNET_ddb_fas_cummul_highfreq_remove.pth', map_location='cpu')
# no_img_df = {'ddbs': data['ddbs'], 'FAS_score': data['FAS_score'], 'steps_to_atk': data['steps_to_atk'], 'predictions': data['predictions'] }
df = pd.DataFrame(data)
df['ddbs'] = list(map(lambda x:float(x), df['ddbs']))

store_results = OrderedDict()
met = [metrics.explained_variance_score, metrics.max_error, metrics.mean_absolute_error, metrics.mean_squared_error, metrics.mean_squared_log_error, metrics.median_absolute_error, metrics.r2_score, metrics.mean_poisson_deviance, metrics.mean_gamma_deviance, metrics.mean_absolute_percentage_error]
for m in met:
    store_results[m.__name__] = []

for loss in met:
    try:

        ## Input: steps_to_atk
        ## Output: lip

        # X = df['steps_to_atk'].values.reshape(-1,1)
        # y = df['steps_to_atk']

        # X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        # reg = linear_model.LinearRegression()
        # reg.fit(X_train, y_train)
        # y_pred = reg.predict(X_test)
        # store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: ddbs
        ## Output: Lip

        X = df['ddbs'].values.reshape(-1,1)
        y = df['steps_to_atk']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: Flipping_Freq
        ## Output: lip

        X = df['Flipping_Freq'].values.reshape(-1,1)
        y = df['steps_to_atk']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)

        store_results[loss.__name__].append(loss(y_test,y_pred))


        ## Input: ddbs, Flipping_Freq
        ## Output: lip

        X = df[['ddbs','Flipping_Freq']]
        y = df['steps_to_atk']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

        ## Input: steps_to_atk, Flipping_Freq
        ## Output: lip

        X = df[['steps_to_atk','Flipping_Freq']]
        y = df['steps_to_atk']

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=0,)

        reg = linear_model.LinearRegression()
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        store_results[loss.__name__].append(loss(y_test,y_pred))

    except Exception as e:
        del store_results[loss.__name__]
        # print(f'skipped loss: {loss.__name__}')
        
store_results = pd.DataFrame(store_results)
store_results.index = ['ddbs vs steps_to_atk', 'Flipping_Freq vs steps_to_atk','ddbs + Flipping_Freq vs steps_to_atk', 'steps_to_atk + Flipping_Freq vs steps_to_atk']
store_results

,explained_variance_score,max_error,mean_absolute_error,mean_squared_error,median_absolute_error,r2_score,mean_absolute_percentage_error
ddbs vs steps_to_atk,0.015185,1.486321e+01,2.235678e-01,1.613938e-01,1.352535e-01,0.015172,2.943872e+12
Flipping_Freq vs steps_to_atk,0.028561,1.476487e+01,2.184804e-01,1.592033e-01,1.216711e-01,0.028538,2.885812e+12
ddbs + Flipping_Freq vs steps_to_atk,0.053011,1.473404e+01,2.186699e-01,1.551960e-01,1.305112e-01,0.052991,2.725411e+12
steps_to_atk + Flipping_Freq vs steps_to_atk,1.000000,5.790923e-13,8.755812e-15,2.458986e-28,4.884981e-15,1.000000,1.183097e-01
